# Stanford University: Tensorflow for Deep Learning Research
## Syllabus and Lecture Notes [http://web.stanford.edu/class/cs20si/syllabus.html](http://web.stanford.edu/class/cs20si/syllabus.html)  
## Code Examples https://github.com/chiphuyen/stanford-tensorflow-tutorials

# Lecture 1 Graphs and Sessions

In [1]:
import tensorflow as tf

<img src="picture/add.jpg" width="30%">

In [2]:
a = tf.add(3,5)

In [3]:
a

<tf.Tensor 'Add:0' shape=() dtype=int32>

In [4]:
a=tf.add(3,5)

sess = tf.Session()
print(sess.run(a))
sess.close()

with tf.Session() as sess:
    print(sess.run(a))

8
8


<img src="picture/graph1.jpg" width="50%">

In [5]:
x = 2
y = 3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)
op3 = tf.pow(op2,op1)
with tf.Session() as sess:
    op3 = sess.run(op3)
    print(op3)

7776


<img src="picture/graph2.jpg" width="50%">

In [6]:
x = 2
y = 3
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)
with tf.Session() as sess:
    z = sess.run(pow_op)
    print(z)

15625


In [7]:
x = 2
y = 3
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)
with tf.Session() as sess:
    z, not_useless = sess.run([pow_op, useless])
    print(z, not_useless)
# tf.Session.run(fetches,feed_dict=None,options=None,run_metadata=None)
# fetches is a list of tensors whose values you want

15625 10


In [8]:
import os
from tensorflow.python.client import device_lib
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "99"
 
if __name__ == "__main__":
    print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14562712191971591582
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6708889518
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3363042787291213470
physical_device_desc: "device: 0, name: Quadro P4000, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [9]:
with tf.device('/device:GPU:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='b')
    c = tf.multiply(a,b)
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    print(sess.run(c))

[[ 1.  4.  9.]
 [16. 25. 36.]]


In [10]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
# Runs the op.
print(sess.run(c))
sess.close()

[[22. 28.]
 [49. 64.]]


In [11]:
# Creates a graph.
c = []
for d in ['/device:GPU:0', '/device:CPU:0']:
    with tf.device(d):
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
        c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
    sum = tf.add_n(c)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(sum))
sess.close()

[[ 44.  56.]
 [ 98. 128.]]


---

# Lecture 2 TensorFlow Ops

In [12]:
import tensorflow as tf

# tf.summary.FileWriter会把当前环境所有的变量和图都画出来, 因此在定义变量之前先把graph reset
tf.reset_default_graph()

a=tf.constant(2)
b=tf.constant(3)
x=tf.add(a,b)
writer=tf.summary.FileWriter('./graphs',tf.get_default_graph()) #生成的events文件存放在graphs文件夹
with tf.Session() as sess:
    print(sess.run(x))
writer.close() # close the writer when you're done using it

5


**tensorbord visualization**  
1. open **cmd**
2. tensorboard --logdir=D:\Ubuntu\UbuntuShare\Jupyter_Notebooks\Stanford_CS20SI--TensorFlow-for-Deep-Learning-Research\graphs
3. open **[http://localhost:6006](http://localhost:6006)** in your browser

In [13]:
import tensorflow as tf

# tf.summary.FileWriter会把当前环境所有的变量和图都画出来
tf.reset_default_graph()

a = tf.constant(2, name='a')
b = tf.constant(3, name='b')
x = tf.add(a, b, name='add')

with tf.Session() as sess:
    tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(x)) # >> 5

5


## 矩阵对应元素相乘 tf.multiply，维度不一致时进行广播

In [14]:
import tensorflow as tf

# tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)
# Broadcasting similar to NumPy
a = tf.constant([2, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')
x1 = tf.multiply(a, b, name='mul1')
x2 = tf.multiply(b, a, name='mul2')
with tf.Session() as sess:
    print(sess.run(x1))
    print(sess.run(x2))

[[0 2]
 [4 6]]
[[0 2]
 [4 6]]


## 矩阵点乘 tf.matmul

In [15]:
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3])
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2])
c = tf.matmul(a, b)
with tf.Session() as sess:
    print(sess.run(c))

[[ 58  64]
 [139 154]]


In [16]:
import numpy as np
a = tf.constant(np.arange(1, 13, dtype=np.int32), shape=[2, 2, 3])
b = tf.constant(np.arange(13, 25, dtype=np.int32), shape=[2, 3, 2])
c = tf.matmul(a, b)
with tf.Session() as sess:
    print(sess.run(c))

[[[ 94 100]
  [229 244]]

 [[508 532]
  [697 730]]]


## Constants

In [17]:
# tf.zeros(shape, dtype=tf.float32, name=None)
# creates a tensor of shape and all elements will be zeros
# Similar to numpy.zeros
zeros = tf.zeros([2, 3], tf.int32)
with tf.Session() as sess:
    print(sess.run(tf.zeros_like(zeros)))

[[0 0 0]
 [0 0 0]]


In [18]:
# tf.zeros_like(input_tensor, dtype=None, name=None, optimize=True)
# creates a tensor of shape and type (unless type is specified) as the input_tensor but all elements are zeros.
# Similar to numpy.zeros_like
input_tensor=[[0, 1], [2, 3], [4, 5]]
with tf.Session() as sess:
    print(sess.run(tf.zeros_like(input_tensor)))

[[0 0]
 [0 0]
 [0 0]]


In [19]:
# Similar to numpy.ones, numpy.ones_like
# tf.ones(shape, dtype=tf.float32, name=None)
# tf.ones_like(input_tensor, dtype=None, name=None, optimize=True)

In [20]:
# tf.fill(dims, value, name=None)
# creates a tensor filled with a scalar value.
# Similar to NumPy.full
with tf.Session() as sess:
    print(sess.run(tf.fill([2, 3], 8)))

[[8 8 8]
 [8 8 8]]


In [21]:
# tf.lin_space(start, stop, num, name=None)
with tf.Session() as sess:
    print(sess.run(tf.lin_space(10.0, 13.0, 4)))

[10. 11. 12. 13.]


In [22]:
# tf.range(start, limit=None, delta=1, dtype=None, name='range')
# NOT THE SAME AS NUMPY SEQUENCES, Tensor objects are not iterable
# for _ in tf.range(4): # TypeError
with tf.Session() as sess:
    print(sess.run(tf.range(3, 18, 3)))
    print(sess.run(tf.range(5)))

[ 3  6  9 12 15]
[0 1 2 3 4]


In [23]:
# Randomly Generated Constants
# tf.random_normal
# tf.truncated_normal
# tf.random_uniform
# tf.random_shuffle
# tf.random_crop
# tf.multinomial
# tf.random_gamma
# tf.set_random_seed(seed)

## Operations

<img src="picture/Operations.jpg" width="90%">

## Arithmetic Ops

<img src="picture/arithmetic.jpg" width="25%">

## Wizard of Div

In [24]:
a = tf.constant([2, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')
with tf.Session() as sess:
    print(sess.run(tf.div(b, a)))
    print(sess.run(tf.divide(b, a)))
    print(sess.run(tf.truediv(b, a)))
    print(sess.run(tf.floordiv(b, a)))
    #print(sess.run(tf.realdiv(b, a))) # Error: only works for real values
    print(sess.run(tf.truncatediv(b, a)))
    print(sess.run(tf.floor_div(b, a)))

[[0 0]
 [1 1]]
[[0.  0.5]
 [1.  1.5]]
[[0.  0.5]
 [1.  1.5]]
[[0 0]
 [1 1]]
[[0 0]
 [1 1]]
[[0 0]
 [1 1]]


## TensorFlow Data Types

In [25]:
# TensorFlow takes Python natives types: boolean, numeric (int, float), strings

t_0 = 19  # scalars are treated like 0-d tensors
with tf.Session() as sess:
    print(sess.run(tf.zeros_like(t_0)))
    print(sess.run(tf.ones_like(t_0)))

t_1 = [b"apple", b"peach", b"grape"] # 1-d arrays are treated like 1-d tensors
with tf.Session() as sess:
    print(sess.run(tf.zeros_like(t_1)))
    #print(sess.run(tf.ones_like(t_1))) #TypeError: Expected string, got 1 of type 'int' instead.

t_2 = [[True, False, False],[False, False, True],[False, True, False]]
with tf.Session() as sess:
    print(sess.run(tf.zeros_like(t_2)))
    print(sess.run(tf.ones_like(t_2)))

0
1
[b'' b'' b'']
[[False False False]
 [False False False]
 [False False False]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


<img src="picture/datatype.jpg" width="30%">

## TF vs NP Data Types

In [26]:
# TensorFlow integrates seamlessly with NumPy
tf.int32 == np.int32

True

In [27]:
# Can pass numpy types to TensorFlow ops
with tf.Session() as sess:
    print(sess.run(tf.ones([2, 2], np.float32)))

[[1. 1.]
 [1. 1.]]


In [28]:
# For tf.Session.run(fetches): if the requested fetch is a Tensor, 
# output will be a NumPy ndarray.
sess = tf.Session()
a = tf.zeros([2, 3], np.int32)
print(type(a))
a_out = sess.run(a) # Avoid doing using a = sess.run(a)
print(type(a_out))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.ndarray'>


**What’s wrong with constants?**  
1. Constants are stored in the graph definition  
2. This makes loading graphs expensive when constants are big  

**advice:**
1. Only use constants for primitive types.  
2. Use variables or readers for more data that requires more memory

In [29]:
import tensorflow as tf
my_const = tf.constant([1.0, 2.0], name="my_const")
#with tf.Session() as sess:
    #print(sess.graph.as_graph_def())

## Variables

In [30]:
# create variables with tf.Variable
s = tf.Variable(2, name="scalar")
m = tf.Variable([[0, 1], [2, 3]], name="matrix")
W = tf.Variable(tf.zeros([784,10]))

# create variables with tf.get_variable
s = tf.get_variable("scalar", initializer=tf.constant(2))
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

**tf.Variable holds several ops:**
```python
x = tf.Variable(...)
x.initializer        # init op
x.value()            # read op
x.assign(...)        # write op
x.assign_add(...)    # and more
```

## initialize your variables

In [31]:
# The easiest way is initializing all variables at once:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
# Initialize only a subset of variables:
with tf.Session() as sess:
    sess.run(tf.variables_initializer([s, m]))
    
# Initialize a single variable
W = tf.Variable(tf.zeros([784,10]))
with tf.Session() as sess:
    sess.run(W.initializer)

## Eval() a variable

In [32]:
# W is a random 700 x 100 variable object
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)    # Similar to print(sess.run(W))
    print(W.eval())

[[ 1.2422355  -0.3917434  -0.11916821 ... -0.87804025 -0.16191287
  -0.55864793]
 [-0.98482686 -0.965423   -0.43619162 ... -0.17281497  0.68641764
  -0.4503836 ]
 [ 0.253377   -0.2546041  -0.6354902  ...  0.757269   -0.49024028
  -0.8581709 ]
 ...
 [ 0.85653347 -1.3462121   0.11082723 ...  1.155268   -1.1771042
   1.6396282 ]
 [ 0.5885185  -1.3611814   0.8452932  ... -0.86036456 -1.079236
  -0.39383748]
 [ 0.44654584  1.2938877   1.0985478  ...  0.28670418  0.9230725
  -0.8499177 ]]


In [33]:
W = tf.Variable(10)
W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval())
    
# W.assign(100) creates an assign op.That op needs to be executed in a session to take effect.
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    sess.run(assign_op)
    print(W.eval())

10
100


In [34]:
# create a variable whose original value is 2
my_var = tf.Variable(2, name="my_var")
# assign a * 2 to a and call that op a_times_two
# It assign 2 * my_var to my_var every time my_var_times_two op is executed.
my_var_times_two = my_var.assign(2 * my_var)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    print(sess.run(my_var_times_two))
    print(sess.run(my_var_times_two))
    print(sess.run(my_var_times_two))

4
8
16


## assign_add() and assign_sub()

In [35]:
my_var = tf.Variable(10)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    # increment by 10
    print(sess.run(my_var.assign_add(10)))
    # decrement by 2
    print(sess.run(my_var.assign_sub(2)))

20
18


## Each session maintains its own copy of variables

In [36]:
W = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10)))
print(sess2.run(W.assign_sub(2)))

print(sess1.run(W.assign_add(100)))
print(sess2.run(W.assign_sub(50)))

sess1.close()
sess2.close()

20
8
120
-42


**A TF program often has 2 phases:**
1. Assemble a graph
2. Use a session to execute operations in the graph.  

⇒ Assemble the graph first without knowing the values needed for computation  

**Analogy:**  
Define the function $f(x, y) = 2*x + y$ without knowing value of $x$ or $y$.  
$x$, $y$ are placeholders for the actual values.

## Placeholders
```python
tf.placeholder(dtype, shape=None, name=None)
```  

**注意：**
1. shape=None means that tensor of any shape will be accepted as value for placeholder.
2. shape=None is easy to construct graphs, but nightmarish for debugging
3. shape=None also breaks all following shape inference, which makes many ops not work because they expect certain rank.

In [37]:
# tf.placeholder(dtype, shape=None, name=None)

# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])

# create a constant of type float 32-bit, shape is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b # Short for tf.add(a, b)

with tf.Session() as sess:
    print(sess.run(c, {a: [1, 2, 3]}))

[6. 7. 8.]


You can feed_dict any feedable tensor. Placeholder is just a way to indicate that something must be fed  
```python
tf.Graph.is_feedable(tensor)
```  
True if and only if tensor is feedable.

<img src="picture/placeholder.jpg" width="30%">

In [38]:
# create operations, tensors, etc (using the default graph)
a = tf.add(2, 5)
b = tf.multiply(a, 3)
with tf.Session() as sess:
# compute the value of b given a is 15
    print(sess.run(b, feed_dict={a: 15}))

45


**Normal loading**

In [39]:
tf.reset_default_graph()
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y) # create the node before executing the graph
writer = tf.summary.FileWriter('./graphs/normal_loading', tf.get_default_graph())  # event文件存放在normal_loading文件夹中
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(z)
writer.close()

**Lazy loading**  
```Defer creating/initializing an object until it is needed```

In [40]:
tf.reset_default_graph()
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
#writer = tf.summary.FileWriter('./graphs/lazy_loading', tf.get_default_graph())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(tf.add(x, y)) # someone decides to be clever to save one line of code
    writer = tf.summary.FileWriter('./graphs/lazy_loading', sess.graph)
#writer.close()

In [41]:
# 查看graph的详细结构和属性
# tf.get_default_graph().as_graph_def()

通过tensorboard查看graph，可以看到：Normal loading情况下，节点"Add"在graph上只定义了1次；而Lazy loading情况下，节点"Add"在graph上只定义了10次  

偷懒模式，节省了代码行数，但是增加了计算次数，使网络变得臃肿缓慢  

**Normal loading**
```python
node {
  name: "Add"
  op: "Add"
  input: "x/read"
  input: "y/read"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
```

**Lazy loading**
```python
node {
  name: "Add_1"
  op: "Add"
  ...
}
...
node {
  name: "Add_10"
  op: "Add"
  ...
}
```

---

**插入图片的两种方式，第一种调整不了显示尺寸**
```
![title](picture/add.jpg)

<img src="picture/add.jpg" width="20%">
```